<a href="https://colab.research.google.com/github/MRMRAC/NN/blob/main/HEARTJATTACK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
import pandas as pd

# Загрузить файл с компьютера
uploaded = files.upload()

Saving cardio_train.csv to cardio_train.csv


In [ ]:
import pandas as pd

# Загрузи файлы вручную через интерфейс Colab:
# Files -> Upload files
# Или:
# from google.colab import files
# uploaded = files.upload()

# 1. Чтение всех датасетов
cardio = pd.read_csv("cardio_train.csv", sep=';')
heart_failure = pd.read_csv("heart_failure_clinical_records_dataset.csv")
heart = pd.read_csv("heart.csv")

# 2. Приведение к общим признакам
cardio.rename(columns={
    'age': 'age_days',
    'gender': 'sex',
    'ap_hi': 'ap_hi',
    'ap_lo': 'ap_lo',
    'cholesterol': 'cholesterol',
    'cardio': 'target'
}, inplace=True)

# Преобразуем возраст в года
cardio['age'] = (cardio['age_days'] / 365).astype(int)

# Для heart_failure
heart_failure.rename(columns={
    'age': 'age',
    'sex': 'sex',
    'creatinine_phosphokinase': 'cpk',
    'serum_creatinine': 'creatinine',
    'serum_sodium': 'serum_sodium',
    'DEATH_EVENT': 'target'
}, inplace=True)

# Для heart
heart.rename(columns={'target': 'target'}, inplace=True)

# 3. Приведение к общему набору столбцов
common_cols = ['age', 'sex', 'cholesterol', 'ap_hi', 'ap_lo',
               'creatinine', 'serum_sodium', 'target']

cardio_part = cardio.reindex(columns=common_cols)
heart_failure_part = heart_failure.reindex(columns=common_cols)
heart_part = heart.reindex(columns=common_cols)

# 4. Объединение
combined = pd.concat([cardio_part, heart_failure_part, heart_part], ignore_index=True)

# 5. Быстрая проверка
print(combined.shape)
print(combined.head())

# 6. (по желанию) сохранить результат
combined.to_csv("combined.csv", index=False)


(71324, 8)
    age  sex  cholesterol  ap_hi  ap_lo  creatinine  serum_sodium  target
0  50.0    2          1.0  110.0   80.0         NaN           NaN       0
1  55.0    1          3.0  140.0   90.0         NaN           NaN       1
2  51.0    1          3.0  130.0   70.0         NaN           NaN       1
3  48.0    2          1.0  150.0  100.0         NaN           NaN       1
4  47.0    1          1.0  100.0   60.0         NaN           NaN       0


In [ ]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, accuracy_score
import numpy as np



In [ ]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"

# === 1. Данные ===
data = pd.read_csv("combined.csv")
data = data.dropna(axis=1, how='all')
data = data.fillna(data.median())

y = data["target"].values
X = data.drop(columns=["target"]).values

scaler = StandardScaler()
X = scaler.fit_transform(X)




In [ ]:
# === 2. Разделение на train/val/test ===
X_train_full, X_test, y_train_full, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full, test_size=0.1, stratify=y_train_full, random_state=42
)

# === 3. Тензоры и DataLoader ===
X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.long)
X_val_t = torch.tensor(X_val, dtype=torch.float32)
y_val_t = torch.tensor(y_val, dtype=torch.long)

train_ds = TensorDataset(X_train_t, y_train_t)
valid_ds = TensorDataset(X_val_t, y_val_t)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_ds, batch_size=32)

# === 4. Модель ===
class NewClassifier(nn.Module):
    def __init__(self, n_inputs, n_hidden1, n_hidden2, n_classes):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(n_inputs, n_hidden1),
            nn.ReLU(),
            nn.Linear(n_hidden1, n_hidden2),
            nn.ReLU(),
            nn.Linear(n_hidden2, n_classes)
        )

    def forward(self, X):
        return self.mlp(X)

torch.manual_seed(42)
n_features = X_train.shape[1]
n_classes = len(np.unique(y))
model = NewClassifier(n_inputs=n_features, n_hidden1=64, n_hidden2=32, n_classes=n_classes).to(device)

# === 5. Optimizer и Loss ===
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
criterion = nn.CrossEntropyLoss()

# === 6. Функции для метрики (Accuracy) ===
def accuracy_metric(y_pred, y_true):
    y_pred_labels = y_pred.argmax(dim=1)
    return (y_pred_labels == y_true).float().mean().item()

def evaluate(model, data_loader):
    model.eval()
    total_correct = 0
    total_samples = 0
    with torch.no_grad():
        for X_batch, y_batch in data_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            y_pred = model(X_batch)
            total_correct += (y_pred.argmax(dim=1) == y_batch).sum().item()
            total_samples += y_batch.size(0)
    return total_correct / total_samples



In [ ]:
# === 7. Функция train ===
def train(model, optimizer, criterion, train_loader, valid_loader, n_epochs):
    history = {"train_loss": [], "train_acc": [], "val_acc": []}
    for epoch in range(n_epochs):
        model.train()
        total_loss = 0
        total_correct = 0
        total_samples = 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            total_correct += (y_pred.argmax(dim=1) == y_batch).sum().item()
            total_samples += y_batch.size(0)
        train_loss = total_loss / len(train_loader)
        train_acc = total_correct / total_samples
        val_acc = evaluate(model, valid_loader)

        history["train_loss"].append(train_loss)
        history["train_acc"].append(train_acc)
        history["val_acc"].append(val_acc)

        print(f"Epoch {epoch+1}/{n_epochs} | "
              f"Train Loss: {train_loss:.4f} | "
              f"Train Acc: {train_acc:.4f} | "
              f"Val Acc: {val_acc:.4f}")
    return history



In [ ]:
# === 8. Обучение ===
history = train(model, optimizer, criterion, train_loader, valid_loader, n_epochs=20)

# === 9. Финальный Test Accuracy ===
X_test_t = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_t = torch.tensor(y_test, dtype=torch.long).to(device)
model.eval()
with torch.no_grad():
    y_pred_test = model(X_test_t)
    test_acc = (y_pred_test.argmax(dim=1) == y_test_t).float().mean().item()
print(f"\n✅ Test Accuracy: {test_acc:.4f}")